# Team Operations : Reset, Stop, Resume and Abort

In [1]:
import asyncio
from autogen_ext.models.openai import OpenAIChatCompletionClient
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')
model_client = OpenAIChatCompletionClient(model='gpt-4o', api_key=api_key)

In [2]:
from autogen_agentchat.agents import AssistantAgent
add_1_agent_first = AssistantAgent(
    name = 'add_1_agent_first',
    model_client=model_client,
    system_message="Add 1 to the number, first number is 0. Give result as output"
)

add_1_agent_second = AssistantAgent(
    name = 'add_1_agent_second',
    model_client=model_client,
    system_message="Add 1 to the number. Give result as output."
)
 
add_1_agent_third = AssistantAgent(
    name = 'add_1_agent_third',
    model_client=model_client,
    system_message="Add 1 to the number. Give result as output."
)


In [3]:
from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    [add_1_agent_first, add_1_agent_second, add_1_agent_third],
    max_turns=3
)

In [4]:
from autogen_agentchat.ui import Console

await Console(team.run_stream())

---------- add_1_agent_first ----------
The initial number is 0. Adding 1 to it gives a result of 1.
---------- add_1_agent_second ----------
The initial number is 1. Adding 1 to it gives a result of 2.
---------- add_1_agent_third ----------
The result of adding 1 to the number 1 is 2.


TaskResult(messages=[TextMessage(source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=24, completion_tokens=20), metadata={}, content='The initial number is 0. Adding 1 to it gives a result of 1.', type='TextMessage'), TextMessage(source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=47, completion_tokens=20), metadata={}, content='The initial number is 1. Adding 1 to it gives a result of 2.', type='TextMessage'), TextMessage(source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=75, completion_tokens=16), metadata={}, content='The result of adding 1 to the number 1 is 2.', type='TextMessage')], stop_reason='Maximum number of turns 3 reached.')

# Resuming a Team
Teams are stateful and maintains the conversation history and context after each run, unless you reset the team.


We can resume a team to continue from where it left off by calling the run() or run_stream() method without a **new task**


In [36]:
await Console(team.run_stream())

---------- add_1_agent_first ----------
4
---------- add_1_agent_second ----------
5
---------- add_1_agent_third ----------
6


TaskResult(messages=[TextMessage(source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=55, completion_tokens=2), metadata={}, content='4', type='TextMessage'), TextMessage(source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=60, completion_tokens=2), metadata={}, content='5', type='TextMessage'), TextMessage(source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=70, completion_tokens=2), metadata={}, content='6', type='TextMessage')], stop_reason='Maximum number of turns 3 reached.')

In [37]:
await Console(team.run_stream())

---------- add_1_agent_first ----------
7
---------- add_1_agent_second ----------
8
---------- add_1_agent_third ----------
9


TaskResult(messages=[TextMessage(source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=86, completion_tokens=2), metadata={}, content='7', type='TextMessage'), TextMessage(source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=91, completion_tokens=2), metadata={}, content='8', type='TextMessage'), TextMessage(source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=101, completion_tokens=2), metadata={}, content='9', type='TextMessage')], stop_reason='Maximum number of turns 3 reached.')

# team resumed from where it left off in the output above, and the first message is from the next agent after the last agent that spoke before the team stopped.

In [38]:
await Console(team.run_stream(task = 'What was the largest number you got in the result?'))

---------- user ----------
What was the largest number you got in the result?
---------- add_1_agent_first ----------
The largest number I provided in the result was 9.
---------- add_1_agent_second ----------
The largest number I provided in the result was 10.
---------- add_1_agent_third ----------
The largest number provided in the result was 9.


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='What was the largest number you got in the result?', type='TextMessage'), TextMessage(source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=133, completion_tokens=13), metadata={}, content='The largest number I provided in the result was 9.', type='TextMessage'), TextMessage(source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=149, completion_tokens=13), metadata={}, content='The largest number I provided in the result was 10.', type='TextMessage'), TextMessage(source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=170, completion_tokens=12), metadata={}, content='The largest number provided in the result was 9.', type='TextMessage')], stop_reason='Maximum number of turns 3 reached.')

# Reset our Team

In [39]:
await team.reset() # on_reset() on all agents

In [40]:
await Console(team.run_stream())

---------- add_1_agent_first ----------
1
---------- add_1_agent_second ----------
2
---------- add_1_agent_third ----------
3


TaskResult(messages=[TextMessage(source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=24, completion_tokens=2), metadata={}, content='1', type='TextMessage'), TextMessage(source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=29, completion_tokens=2), metadata={}, content='2', type='TextMessage'), TextMessage(source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=39, completion_tokens=2), metadata={}, content='3', type='TextMessage')], stop_reason='Maximum number of turns 3 reached.')

## Covered in Future Videos in the Module

# Aborting a Team

Different from stopping a team, aborting a team will immediately stop the team and raise a CancelledError exception.

In [ ]:
from autogen_core import CancellationToken

cancellation_token = CancellationToken()

run2 = asyncio.create_task(
    Console(team.run_stream(task = 'Give a short Story about a lion atmost 40 words',cancellation_token=cancellation_token))
)

await asyncio.sleep(2)
cancellation_token.cancel()

try:
    result = await run2
except asyncio.CancelledError():
    print("Task Was Cancelled")

TypeError: catching classes that do not inherit from BaseException is not allowed